In [1]:
import os, sys
sys.path.append(os.path.abspath(".."))

import requests
import scrapy
import time
import pandas as pd
from pprint import pprint
import re
import json
import datetime

from scrapers import staff

In [2]:
data = staff.main()

Sat Dec 25 20:52:34 2021
Starting crawling 'https://staff.am'.
1221 jobs detected.
Starting crawling job postings..


URL https://staff.am/en/onlayn-xagavar-29: 100%|██████████| 1221/1221 [41:51<00:00,  2.06s/it]                                                                                         


1221/1221 jobs scraped.
Saving files.
Company_Title: 1221
Company_URL: 1221
URL: 1221
Total_views: 1221
Followers: 1221
Active_Jobs: 1221
Jobs_History: 1221
Job_Views: 1221
Job_Title: 1221
Application_Deadline: 1221
Other: 1221
Industry: 1221
Employment_term: 1221
Job_Category: 1221
Job_type: 1221
Job_Location: 1221
Text: 1221
Job_description: 1221
Job_responsibilities: 1221
Required_qualifications: 1221
Required_candidate_level: 1221
Salary: 1221
Additional_information: 1221
Professional_skills: 1221
Soft_skills: 1221
Detected 4 new companies.
Starting crawling companies.


100%|██████████| 4/4 [00:02<00:00,  1.42it/s]

Extracted 4/4
Sat Dec 25 21:34:53 2021
Ended crawling 'https://staff.am'.


In [ ]:
with open("../data/postings/staff_08.10.21.json", "r") as f:
    dat = json.load(f)

len(dat["URL"])


In [ ]:
len(data.keys())

In [ ]:
all_default_keys = (
    "Company_Title", "Company_URL", "URL", "Total_views", "Followers",
    "Active_Jobs", "Jobs_History", "Job_Views", "Job_Title",
    "Application_Deadline", "Industry", "Employment_term",
    "Job_Category", "Job_type", "Job_Location",
    "Job_description", "Job_responsibilities",
    "Required_qualifications", "Required_candidate_level",
    "Salary", "Additional_information",
    "Professional_skills", "Soft_skills"
)

In [ ]:
new_keys = [k for k in data.keys() if k not in all_default_keys]

In [ ]:
len(new_keys)

In [ ]:
sorted([(k, len(data[k])) for k in new_keys], key=lambda x: x[1])[::-1]

In [ ]:
data["Additional_information"]

In [ ]:
rs = requests.get("https://staff.am/en/senior-uiux-designer-112")
response = scrapy.http.HtmlResponse(url=rs.url, body=rs.text, encoding="utf-8")

In [ ]:
staff.get_info_from_one_posting("https://staff.am/en/senior-uiux-designer-112")

In [ ]:
scrape = staff.crawl_all_postings(["https://staff.am/en/senior-engineer-ii-c", "https://staff.am/en/senior-uiux-designer-112"])

In [ ]:
arr = [f"{i}th val" for i in range(100)]

In [ ]:
import time
from rich.progress import track
from rich import progress_bar

In [ ]:
for i in track(arr):
    time.sleep(0.1)